In [447]:
import requests
import urllib
import json
import pandas as pd

In [448]:
tokens = pd.read_csv(r'C:\Users\mason\Desktop\Twitter Keys.txt', sep = '\t', header = None).T
tokens.columns = tokens.iloc[0]
tokens = tokens[1:]
#tokens.Bearer_Token[1]

In [449]:
def auth():
    return tokens.Bearer_Token[1]

#print(auth())

In [450]:
def create_url():
    query = urllib.parse.quote('(BLM) OR #BLM -is:retweet lang:en')
    tweet_fields = 'tweet.fields=author_id,conversation_id,public_metrics,id,text,created_at'
    url = 'https://api.twitter.com/2/tweets/search/recent?query={}&{}'.format(query, tweet_fields,)
    return url

print(create_url())

https://api.twitter.com/2/tweets/search/recent?query=%28BLM%29%20OR%20%23BLM%20-is%3Aretweet%20lang%3Aen&tweet.fields=author_id,conversation_id,public_metrics,id,text,created_at


In [451]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

#print(create_headers(token.Bearer_Token[1]))

In [452]:
def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [453]:
def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    #print(json.dumps(json_response, indent = 4, sort_keys = True))
    return json_response

In [454]:
main()

200


{'data': [{'created_at': '2021-04-29T03:50:20.000Z',
   'author_id': '1360025996569812992',
   'id': '1387615206763909122',
   'public_metrics': {'retweet_count': 19,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': 'RT @ScoonTv: Groups chaired by BLM co-founder plagued by financial controversies https://t.co/Fh8ubIZpBc',
   'conversation_id': '1387615206763909122'},
  {'created_at': '2021-04-29T03:50:18.000Z',
   'author_id': '260909624',
   'id': '1387615198350094342',
   'public_metrics': {'retweet_count': 73,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': 'RT @ElAmerican_: Destruyendo la Policía: renuncia masiva de oficiales en #NewYork\n\n#BLM, izquierda, demócratas extremos, están logrando aca…',
   'conversation_id': '1387615198350094342'},
  {'created_at': '2021-04-29T03:50:16.000Z',
   'author_id': '224072929',
   'id': '1387615188652756998',
   'public_metrics': {'retweet_count': 29,
    'reply_count': 0,
    'like_cou

In [455]:
my_twitter_data = main()

200


In [456]:
my_twitter_data.keys()

dict_keys(['data', 'meta'])

In [457]:
my_twitter_data['data'][0]

{'created_at': '2021-04-29T03:50:20.000Z',
 'id': '1387615206763909122',
 'conversation_id': '1387615206763909122',
 'author_id': '1360025996569812992',
 'public_metrics': {'retweet_count': 19,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'text': 'RT @ScoonTv: Groups chaired by BLM co-founder plagued by financial controversies https://t.co/Fh8ubIZpBc'}

In [458]:
my_twitter_data['meta']

{'newest_id': '1387615206763909122',
 'oldest_id': '1387615170650771459',
 'result_count': 10,
 'next_token': 'b26v89c19zqg8o3fosttqljzcmgchlrefykxkx1yi14zh'}

In [459]:
def create_url_pag(next_token = None):
    query = urllib.parse.quote('(BLM) OR #BLM -is:retweet lang:en')
    tweet_fields = 'tweet.fields=author_id,conversation_id,public_metrics,id,text,created_at'
    if (next_token):
        pagination = next_token
        url = 'https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100&next_token={}&{}'.format(query, pagination, tweet_fields)
        return url
    else:
        query = urllib.parse.quote('(BLM) OR #BLM -is:retweet lang:en')
        tweet_fields = 'tweet.fields=author_id,conversation_id,public_metrics,id,text,created_at'
        url = 'https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100&{}'.format(query, tweet_fields)
        return url
    
print(create_url_pag(None))
print(create_url_pag('000000000'))

https://api.twitter.com/2/tweets/search/recent?query=%28BLM%29%20OR%20%23BLM%20-is%3Aretweet%20lang%3Aen&max_results=100&tweet.fields=author_id,conversation_id,public_metrics,id,text,created_at
https://api.twitter.com/2/tweets/search/recent?query=%28BLM%29%20OR%20%23BLM%20-is%3Aretweet%20lang%3Aen&max_results=100&next_token=000000000&tweet.fields=author_id,conversation_id,public_metrics,id,text,created_at


In [460]:
def multi_page_request():
    count = 0
    next_token = None
    dat = pd.DataFrame()
    while (count < 3):
        bearer_token = auth()
        url = create_url_pag(next_token)
        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(url, headers)
        next_token = json_response['meta']['next_token']
        dat = dat.append(pd.DataFrame(json_response['data']), ignore_index = True)
        
        count = count + 1
    return dat

In [461]:
pages = multi_page_request()

200
200
200


In [462]:
pages

,created_at,author_id,id,public_metrics,text,conversation_id
0,2021-04-29T03:50:20.000Z,1360025996569812992,1387615206763909122,"{'retweet_count': 19, 'reply_count': 0, 'like_...",RT @ScoonTv: Groups chaired by BLM co-founder ...,1387615206763909122
1,2021-04-29T03:50:18.000Z,260909624,1387615198350094342,"{'retweet_count': 73, 'reply_count': 0, 'like_...",RT @ElAmerican_: Destruyendo la Policía: renun...,1387615198350094342
2,2021-04-29T03:50:16.000Z,224072929,1387615188652756998,"{'retweet_count': 29, 'reply_count': 0, 'like_...",RT @tweetmiliter: Banyak yang kemakan omongan ...,1387615188652756998
3,2021-04-29T03:50:16.000Z,720616939060662272,1387615187927191557,"{'retweet_count': 3291, 'reply_count': 0, 'lik...",RT @MrAndyNgo: Just hours after filming hersel...,1387615187927191557
4,2021-04-29T03:50:15.000Z,1328264603952848896,1387615187100868610,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@doonud gatau kita blm mutualan,1387612918594752517
...,...,...,...,...,...,...
295,2021-04-29T03:45:12.000Z,1241009962492411905,1387613916180606976,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@prisma93246654 Mungkin dia blm nikah2 jadi pu...,1387612846108778499
296,2021-04-29T03:45:12.000Z,573307009,1387613914792402946,"{'retweet_count': 15, 'reply_count': 0, 'like_...",RT @TrumpJew2: Fun watching the libs with BLM ...,1387613914792402946
297,2021-04-29T03:45:12.000Z,1334749367643717634,1387613913949249538,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@JackPosobiec FBI has been infiltrated by the ...,1387613451405627392
298,2021-04-29T03:45:11.000Z,92665268,1387613908404367360,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@BossTemlen Sekelas clinton aja klepek2 dipepe...,1387541580135862273


In [463]:
metrics = pd.DataFrame(list(pages['public_metrics']))
metrics

,retweet_count,reply_count,like_count,quote_count
0,19,0,0,0
1,73,0,0,0
2,29,0,0,0
3,3291,0,0,0
4,0,0,0,0
...,...,...,...,...
295,0,0,0,0
296,15,0,0,0
297,0,0,2,0
298,0,0,0,0


In [464]:
full_data = pd.concat([pages.drop("public_metrics", axis = 1).reset_index(drop=True), metrics], axis = 1)
full_data

,created_at,author_id,id,text,conversation_id,retweet_count,reply_count,like_count,quote_count
0,2021-04-29T03:50:20.000Z,1360025996569812992,1387615206763909122,RT @ScoonTv: Groups chaired by BLM co-founder ...,1387615206763909122,19,0,0,0
1,2021-04-29T03:50:18.000Z,260909624,1387615198350094342,RT @ElAmerican_: Destruyendo la Policía: renun...,1387615198350094342,73,0,0,0
2,2021-04-29T03:50:16.000Z,224072929,1387615188652756998,RT @tweetmiliter: Banyak yang kemakan omongan ...,1387615188652756998,29,0,0,0
3,2021-04-29T03:50:16.000Z,720616939060662272,1387615187927191557,RT @MrAndyNgo: Just hours after filming hersel...,1387615187927191557,3291,0,0,0
4,2021-04-29T03:50:15.000Z,1328264603952848896,1387615187100868610,@doonud gatau kita blm mutualan,1387612918594752517,0,0,0,0
...,...,...,...,...,...,...,...,...,...
295,2021-04-29T03:45:12.000Z,1241009962492411905,1387613916180606976,@prisma93246654 Mungkin dia blm nikah2 jadi pu...,1387612846108778499,0,0,0,0
296,2021-04-29T03:45:12.000Z,573307009,1387613914792402946,RT @TrumpJew2: Fun watching the libs with BLM ...,1387613914792402946,15,0,0,0
297,2021-04-29T03:45:12.000Z,1334749367643717634,1387613913949249538,@JackPosobiec FBI has been infiltrated by the ...,1387613451405627392,0,0,2,0
298,2021-04-29T03:45:11.000Z,92665268,1387613908404367360,@BossTemlen Sekelas clinton aja klepek2 dipepe...,1387541580135862273,0,0,0,0


I hope I dont get points off because even though I have an analysis plan I don't 100% know how get it to work yet and I am planning on asking you questions tomorrow. My plan is to use the created_at root field to create 3 queries that request data from three different days, the day after George Floyd's death, 6 months before that, and 6 months after that. The day 6 months before is going to be the control and the analysis is mostly going to focus on how support for the BLM movement has changed from the day after George Floyd's to 6 months later. I am going to determine this by graphing the sentiment analysis ratings from the VADER program for all three days from datasets of around 300 tweets each. After I complete the three graphs I will put the results onto one graph and compare the sentiment analysis ratings and look for noticable differences. The main problems I have right now is how to use created_at to request data from certain dates and how to get the lang:en root field to work as well.